# Preprocessing and Feature Engineering: Customer Churn Data

**Note: Read and write files are markdown to prevent accidentally writing original file over holdout. Remove #'s to output files.**

In [4]:
import pandas as pd
import numpy as np

In [5]:
#df = pd.read_csv('retentiondata_case.csv')
#df = pd.read_csv('retentiondata_case_holdout.csv')

## Initial Examination

In [6]:
df.shape

(5636, 36)

In [7]:
# check target variable distribution
df['left_flag'].value_counts(normalize=True)


left_flag
No     0.734564
Yes    0.265436
Name: proportion, dtype: float64

## Preprocessing

### Removing Columns

**Drop Indentifier Variables**

In [8]:
cols_to_drop = ["acct_ref", "cust_ref"]

df = df.drop(
    columns=[c for c in cols_to_drop if c in df.columns]
)


**Zero-Variance Columns**

In [9]:
# Identify zero-variance columns
zero_var_cols = [
    col for col in df.columns
    if df[col].nunique(dropna=False) == 1
]

zero_var_cols


['fiscal_qtr']

In [10]:
# Drop zero-variance columns
df = df.drop(columns=zero_var_cols)

### Missing Data

In [11]:
# Calculate number and percentage of missing values per column
missing_pct = (
    df.isna()
    .mean()
    .mul(100)
    .round(2)
)

missing_pct[missing_pct > 0]

total_billed      0.14
recent_offer     55.11
internet_tech    21.50
dtype: float64

**Replace Missing Categorical Values**

In [12]:
# Replace missing internet_tech with "Unknown"
if "internet_tech" in df.columns:
    df["internet_tech"] = df["internet_tech"].fillna("Unknown")


# Replace missing recent_offer with "No Offer"
if "recent_offer" in df.columns:
    df["recent_offer"] = df["recent_offer"].fillna("No Offer")


#Quick check
df[["internet_tech", "recent_offer"]].isna().sum()

internet_tech    0
recent_offer     0
dtype: int64

**Replace Missing Numeric Values**

In [13]:
#impute missing values in total_billed with zero
    # all total_billed == 0 are due to tenure_mo == 0   =>   no bills yet

df["total_billed"] = df["total_billed"].fillna(0)

## Feature Engineering

### Overage Ratio

In [14]:
# 1. Overage Ratio: Proportion of current monthly bill coming from 'extra' fees
    # overage ratio = ( total monthly bill - monthly rate ) / monthly rate
    # (actual - expected) / expected

# cases
case_overage = [
    df['tenure_mo'] == 0,
    df['tenure_mo'] != 0    
]
# results
result_overage = [
    ((df['extra_data_fees_total'] + df['avg_long_dist_fee']) / df['monthly_fee']),
    ((df['extra_data_fees_total'] / df['tenure_mo']) + (df['avg_long_dist_fee'])) / (df['monthly_fee'])
]

df['overage_ratio_avg'] = np.select(case_overage, result_overage, default = '').astype(float)

### Add-ons

In [15]:
#Evaluating add-ons

addon_cols = [
    "add_on_backup",
    "add_on_protection",
    "add_on_security",
    "premium_support",
    "tech_support_std",
    "unlimited_data_opt"
]

# 1) Create numeric churn flag (1 = churned)
df["churn_flag"] = (df["left_flag"].astype(str).str.strip().str.lower() == "yes").astype(int)

# 2) Long format for add-ons
churn_long = df.melt(
    id_vars="churn_flag",
    value_vars=[c for c in addon_cols if c in df.columns],
    var_name="add_on",
    value_name="add_on_status"
)

# 3) Pivot: churn % by add-on status
addon_pivot = (
    churn_long
    .groupby(["add_on", "add_on_status"])["churn_flag"]
    .mean()
    .mul(100)
    .unstack()
    .round(2)
)

# 4) Drop "No internet service" column if it exists
if "No internet service" in addon_pivot.columns:
    addon_pivot = addon_pivot.drop(columns="No internet service")

# 5) Add difference column (No - Yes)
addon_pivot["churn_pct_diff"] = (addon_pivot["No"] - addon_pivot["Yes"]).round(2)

addon_pivot


add_on_status,No,Yes,churn_pct_diff
add_on,,,
add_on_backup,39.53,21.89,17.64
add_on_protection,39.37,22.10,17.27
add_on_security,41.76,14.63,27.13
premium_support,31.28,15.12,16.16
tech_support_std,41.65,15.12,26.53
unlimited_data_opt,16.05,31.57,-15.52


In [16]:
# Aggregate Support and Security Add-ons except unlimited data add on

addon_cols = [
    "add_on_backup",
    "add_on_protection",
    "premium_support",
    "add_on_security",
    "tech_support_std"
]

df["num_add_ons"] = (
    df[addon_cols]
    .apply(lambda x: (x == "Yes").sum(), axis=1)
)

In [17]:
# Create col 'has_add_on' indicating customer has support or security add ons

df["has_add_on"] = (df[addon_cols] == "Yes").any(axis=1).astype(int)


In [18]:
# create streaming service aggregation col

stream_cols = [
    "stream_tv",
    "stream_movies",
    "stream_music"
]

df["has_streaming"] = (df[stream_cols] == "Yes").any(axis=1).astype(int)


In [19]:
# create referring behavior aggregation col
df["is_referrer"] = (
    (df["referred_friend"] == "Yes") | (df["referrals_count"] > 0)
).astype(int)


In [20]:
# Create binary refund flag
df["received_refund"] = (
    df["refunds_total"] > 0
).astype(int)

# Drop original refunds_total column
df = df.drop(columns=["refunds_total"])


In [21]:
# Dropping Columns used to create aggregated features
cols_to_drop = [
    # Add-ons (used in num_add_ons / has_add_on)
    "add_on_backup",
    "add_on_protection",
    "add_on_security",
    "premium_support",
    "tech_support_std",

    # Streaming (used in has_streaming)
    "stream_tv",
    "stream_movies",
    "stream_music",

    # Referral (used in is_referrer)
    "referred_friend",
    "referrals_count",
    "internet_plan",
    "dependents_count",

    # Refunds
    "refunds_total",

    # Response
    "churn_flag"
]

# Drop only if columns exist
df = df.drop(
    columns=[c for c in cols_to_drop if c in df.columns]
)

### Export Dataset with Categorical Variables

In [22]:
#df.to_excel('churn_df_data_categorical.xlsx', index=False)
#df.to_excel('churn_df_data_categorical_holdout.xlsx', index=False)

## Create Numeric Dataset

### One-Hot Encode Categorical Columns

In [23]:
# Identify categorical columns
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

cat_cols

['has_dependents',
 'home_phone',
 'multi_line',
 'contract_term',
 'e_bill_opt_in',
 'pay_method',
 'left_flag',
 'gender',
 'is_married',
 'recent_offer',
 'internet_tech',
 'unlimited_data_opt']

In [24]:
# Safely one-hot encode selected categorical columns

encode_cols = [
    col for col in ["pay_method", "internet_tech", "recent_offer", "contract_term", "gender","multi_line"]
    if col in df.columns
]

df = pd.get_dummies(
    df,
    columns=encode_cols,
    drop_first=False   # keep all categories
)

#Quick check
df.shape

(5636, 42)

In [25]:
# Dropping Columns one-hot coded columns
cols_to_drop = ["pay_method", "internet_tech", "recent_offer", "contract_term", "gender","multi_line"]

# Drop only if columns exist
df = df.drop(
    columns=[c for c in cols_to_drop if c in df.columns]
)

In [26]:
# Encode everything EXCEPT the response variable 'left_flag'

response_col = "left_flag"

# 1. Convert boolean dtype columns (True/False) to 1/0
bool_cols = [
    col for col in df.select_dtypes(include="bool").columns
    if col != response_col
]

df[bool_cols] = df[bool_cols].astype(int)

# 2. Convert binary Yes/No and TRUE/FALSE STRING columns to 1/0
binary_string_cols = [
    col for col in df.select_dtypes(include=["object", "string"]).columns
    if col != response_col
    and set(df[col].dropna().unique()).issubset(
        {"Yes", "No", "TRUE", "FALSE"}
    )
]

df[binary_string_cols] = (
    df[binary_string_cols]
    .replace({
        "Yes": 1, "No": 0,
        "TRUE": 1, "FALSE": 0
    })
    .astype(int)
)

# 3. FINAL SAFETY CHECK (response variable may still appear here, which is OK)
[col for col in df.select_dtypes(include=["object", "string"]).columns if col != response_col]

/var/folders/pc/vhr3_b9n23gdj_s0bdp2v9lm0000gn/T/ipykernel_68465/1100615532.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({


[]

In [27]:
# encode target variable
df["left_flag"] = (df["left_flag"] == "Yes").astype(int)

np.isinf(df).sum().sum()

np.int64(0)

In [28]:
# check
df.shape

(5636, 42)

### Export Numeric Dataset

In [29]:
#df.to_excel('churn_df_data_numeric.xlsx', index=False)